In [1]:
%pip install -qU langchain
%pip install -qU huggingface_hub

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
doi_file = '../data/dois.tsv'
with open('assets/X-ELS-APIKEY', 'r') as f:
    elsevier_api_key = f.read()

In [4]:
from scripts.utils import id_convert, get_full_text
import pandas as pd


See which journal articles can be recovered automatically from EuropePMC or Elsevier, the rest will have be downloaded manually

In [5]:
with open(doi_file, 'r') as f:
    dois = set([line.replace("\n", "") for line in f.readlines()])
    f.close()
len(dois)
pmcids = [id_convert(doi) for doi in dois]
ids = pd.DataFrame([dois, pmcids]).transpose()
ids.columns = ['doi', 'pmcid']
# Get full text for pmcids
for index, row in ids.iterrows():
    ids.at[index, 'full_text'] = get_full_text(row['pmcid'], filepath='../data/docs/', doi = row['doi'], elsevier_api_key= elsevier_api_key)
manual_download = ["https://doi.org/"+row['doi'] for index, row in ids.iterrows() if row['full_text'] == ""]
print("Add missing documents (XML or PDF): \n- " + "\n- ".join(manual_download))

Add missing documents (XML or PDF): 
- https://doi.org/10.1039/C2JM31396K
- https://doi.org/10.1021/nn1007176
- https://doi.org/10.1002/smll.201102743
- https://doi.org/10.1021/la203607w
- https://doi.org/10.1021/tx400385x
- https://doi.org/10.1021/nl202515a
- https://doi.org/10.1038/cdd.2010.11
- https://doi.org/10.1021/nn202699t
- https://doi.org/10.1002/smll.201201546
- https://doi.org/10.6023/A20060216
- https://doi.org/10.1021/nn101097v
- https://doi.org/10.1021/am300253c


In [9]:
downloaded = [file.split(".xml")[0] if 'xml' in file else file.split(".pdf")[0] for file in os.listdir('../data/docs')]
dois = [doi.split("/")[-1] for doi in dois]

for doi in dois:
    if doi not in downloaded:
        print(f"Missing document: {doi}")
